# Машинное обучение, ФКН ВШЭ

## Практическое задание 8. Метод опорных векторов и аппроксимация ядер

# ЧАСТЬ 2

### Пришлось разбить на несколько файлов, чтобы иметь возможность обучать параллельно несколько заданий.

Тестировать алгоритм мы будем на данных Fashion MNIST. Ниже код для их загрузки и подготовки.

In [385]:
#!g1.1
import random

import numpy as np
import scipy
from itertools import combinations

from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [386]:
#!g1.1
import keras
from keras.datasets import fashion_mnist
(x_train_pics, y_train), (x_test_pics, y_test) = fashion_mnist.load_data()
x_train = x_train_pics.reshape(x_train_pics.shape[0], -1)
x_test = x_test_pics.reshape(x_test_pics.shape[0], -1)

Using TensorFlow backend.


__Задание 1. (5 баллов)__

Реализуйте алгоритм, описанный выше. Можете воспользоваться шаблоном класса ниже или написать свой интерфейс.

Ваша реализация должна поддерживать следующие опции:
1. Возможность задавать значения гиперпараметров new_dim (по умолчанию 50) и n_features (по умолчанию 1000).
2. Возможность включать или выключать предварительное понижение размерности с помощью метода главных компонент.
3. Возможность выбирать тип линейной модели (логистическая регрессия или SVM с линейным ядром).

Протестируйте на данных Fashion MNIST, сформированных кодом выше. Если на тесте у вас получилась доля верных ответов не ниже 0.84 с гиперпараметрами по умолчанию, то вы всё сделали правильно.

In [402]:
#!c1.8
from sklearn.base import BaseEstimator, TransformerMixin

class RFFPipeline(BaseEstimator, TransformerMixin): 
    """
    При унаследовании от TransformerMixin, fit_transformer() можно не задавать, 
    Если добавить в качестве базового класса BaseEstimator и не реализовывать *args, **kwargs в конструкторе,
    то будут доступны методы get_params() и set_params()
    """
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg'):
        """        
        Implements pipeline, which consists of PCA decomposition,
        Random Fourier Features approximation and linear classification model.
        
        n_features, int: amount of synthetic random features generated with RFF approximation.

        new_dim, int: PCA output size.
        
        use_PCA, bool: whether to include PCA preprocessing.
        
        classifier, string: either 'svm' or 'logreg', a linear classification model to use on top of pipeline.
        
        Feel free to edit this template for your preferences.    
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.normalizer = Normalizer()
        self.new_dim = new_dim
        self.PCA_model = PCA(new_dim)
        self.classifier = classifier
        if self.classifier == 'logreg':
            self.model = LogisticRegression(max_iter = 500, n_jobs = -1, random_state = 72)
        else:
            self.model = SVC(kernel='linear', random_state = 72)
        
    def fit(self, X, y):
        """
        Fit all parts of algorithm (PCA, RFF, Classification) to training set.
        """
        X = self.normalizer.fit_transform(X)
        if self.use_PCA:
            X = self.PCA_model.fit_transform(X, y)
            print('PCA применён')
            
        elements_for_choosing = random.sample(range(X.shape[0]), 1420) # выбираю случ. элементы из выборки для составления пар
        
        dist_list = [] # список пар
        for (c,p) in combinations(elements_for_choosing, 2): # составляю комбинации пар выборки
            dist_list.append(list(scipy.spatial.distance.cdist(X[c].reshape(1,-1), 
                                                               X[p].reshape(1,-1),
                                                               lambda u, v: ((u-v)**2).sum())[0]))
        self.median = np.median(dist_list) # рассчитываю медиану
        print('Медиана посчитана: {:.5f}'.format(self.median))
        self.w = np.random.normal(scale=1./self.median, size=(X.shape[1], self.n_features)) # генерирую набор весов
        self.b = np.random.uniform(-np.pi, np.pi, size=self.n_features) # генерирую набор свдигов
        
        new_features_X = np.cos(np.dot(X, self.w) + self.b)
        self.model.fit(new_features_X, y)
        return
    
    def predict_proba(self, X):
        """
        Apply pipeline to obtain scores for input data.
        """
        X = self.normalizer.transform(X)
        if self.use_PCA:
            X = self.PCA_model.transform(X)
        
        new_features_X = np.cos(np.dot(X, self.w) + self.b)
        return self.model.predict_proba(new_features_X)
        
    def predict(self, X):
        """
        Apply pipeline to obtain discrete predictions for input data.
        """
        X = self.normalizer.transform(X)
        if self.use_PCA:
            X = self.PCA_model.transform(X)
        
        new_features_X = np.cos(np.dot(X, self.w) + self.b)
        return self.model.predict(new_features_X)

### Бонус

__Задание 4. (Максимум 2 балла)__

Как вы, должно быть, помните с курса МО-1, многие алгоритмы машинного обучения работают лучше, если признаки данных некоррелированы. Оказывается, что для RFF существует модификация, позволяющая получать ортогональные случайные признаки (Orthogonal Random Features, ORF). Об этом методе можно прочитать в [статье](https://proceedings.neurips.cc/paper/2016/file/53adaf494dc89ef7196d73636eb2451b-Paper.pdf). Реализуйте класс для вычисления ORF по аналогии с основным заданием. Обратите внимание, что ваш класс должен уметь работать со случаем n_features > new_dim (в статье есть замечание на этот счет). Проведите эксперименты, сравнивающие RFF и ORF, сделайте выводы.

In [444]:
#!c1.8
from sklearn.base import BaseEstimator, TransformerMixin

class ORFPipeline(BaseEstimator, TransformerMixin): 
    """
    При унаследовании от TransformerMixin, fit_transformer() можно не задавать, 
    Если добавить в качестве базового класса BaseEstimator и не реализовывать *args, **kwargs в конструкторе,
    то будут доступны методы get_params() и set_params()
    """
    def __init__(self, n_features=1000, new_dim=50, use_PCA=True, classifier='logreg'):
        """        
        Implements pipeline, which consists of PCA decomposition,
        Random Fourier Features approximation and linear classification model.
        
        n_features, int: amount of synthetic random features generated with RFF approximation.

        new_dim, int: PCA output size.
        
        use_PCA, bool: whether to include PCA preprocessing.
        
        classifier, string: either 'svm' or 'logreg', a linear classification model to use on top of pipeline.
        
        Feel free to edit this template for your preferences.    
        """
        self.n_features = n_features
        self.use_PCA = use_PCA
        self.normalizer = Normalizer()
        self.new_dim = new_dim
        self.PCA_model = PCA(new_dim)
        self.classifier = classifier
        if self.classifier == 'logreg':
            self.model = LogisticRegression(max_iter = 500, n_jobs = -1, random_state = 72)
        else:
            self.model = SVC(kernel='linear', random_state = 72)
        
    def fit(self, X, y):
        """
        Fit all parts of algorithm (PCA, RFF, Classification) to training set.
        """
        X = self.normalizer.fit_transform(X)
        if self.use_PCA:
            X = self.PCA_model.fit_transform(X, y)
            print('PCA применён')
        
        elements_for_choosing = random.sample(range(X.shape[0]), 1420) # выбираю случ. элементы из выборки для составления пар
        dist_list = [] # список пар
        
        n_stacks = int(np.ceil(self.n_features/X.shape[1]))
        
        
        if n_stacks == 1:
            for (c,p) in combinations(elements_for_choosing, 2): # составляю комбинации пар выборки
                dist_list.append(list(scipy.spatial.distance.cdist(X[c, :X.shape[1]].reshape(1,-1), 
                                                                   X[p, :X.shape[1]].reshape(1,-1),
                                                                   lambda u, v: ((u-v)**2).sum())[0]))
            self.median = np.median(dist_list) # рассчитываю медиану
            print('Медиана посчитана: {:.5f}'.format(self.median))
            w = np.random.normal(scale=1./self.median, size=(X.shape[1], self.n_features)) # генерирую набор весов
            s = np.diag(chi.rvs(df = self.n_features, size = X.shape[1]))
            self.w, _ = scipy.linalg.qr_multiply(w, s)
            self.b = np.random.uniform(-np.pi, np.pi, size=self.n_features) # генерирую набор свдигов
        
        else:
            for stck in range(n_stacks):
                if ((stck+1) * X.shape[1]) > self.n_features:
                    for (c,p) in combinations(elements_for_choosing, 2): # составляю комбинации пар выборки
                        dist_list.append(list(scipy.spatial.distance.cdist(X[c, (stck+1) * X.shape[1]:].reshape(1,-1), 
                                                                           X[p, (stck+1) * X.shape[1]:].reshape(1,-1),
                                                                           lambda u, v: ((u-v)**2).sum())[0]))
                else:
                    for (c,p) in combinations(elements_for_choosing, 2): # составляю комбинации пар выборки
                        dist_list.append(list(scipy.spatial.distance.cdist(X[c, stck * X.shape[1]: (stck+1) * X.shape[1]].reshape(1,-1), 
                                                                           X[p, stck * X.shape[1]: (stck+1) * X.shape[1]].reshape(1,-1),
                                                                           lambda u, v: ((u-v)**2).sum())[0]))
                self.median = np.median(dist_list) # рассчитываю медиану
                w = np.random.normal(scale=1./self.median, size=(X.shape[1], self.n_features)) # генерирую набор весов
                s = np.diag(chi.rvs(df = self.n_features, size = X.shape[1]))
                w_new, _ = scipy.linalg.qr_multiply(w, s)
                if stck == 0:
                    self.w = w_new
                else:
                    self.w = np.concatenate([self.w, w_new], axis=1)
            self.b = np.random.uniform(-np.pi, np.pi, size=self.w.shape[1]) # генерирую набор свдигов

        new_features_X = np.cos(np.dot(X, self.w) + self.b)
        self.model.fit(new_features_X, y)
        return
    
    def predict_proba(self, X):
        """
        Apply pipeline to obtain scores for input data.
        """
        X = self.normalizer.transform(X)
        if self.use_PCA:
            X = self.PCA_model.transform(X)
        
        new_features_X = np.cos(np.dot(X, self.w) + self.b)
        return self.model.predict_proba(new_features_X)
        
    def predict(self, X):
        """
        Apply pipeline to obtain discrete predictions for input data.
        """
        X = self.normalizer.transform(X)
        if self.use_PCA:
            X = self.PCA_model.transform(X)
        
        new_features_X = np.cos(np.dot(X, self.w) + self.b)
        return self.model.predict(new_features_X)

### Сравнение качества Orthogonal Random Features и Random Fourier Features на логистической регрессии

In [445]:
#!c1.8
%%time
rffp_kernel = ORFPipeline(n_features=300, new_dim=500)
rffp_kernel.fit(x_train, y_train)
preds = rffp_kernel.predict(x_test)
print('Orthogonal Random Features. Логистическая регрессия при new_dim > n_features. Доля верных ответов на тестовой выборке: {:.5f}\n'.format(accuracy_score(preds, y_test)))

PCA применён
Медиана посчитана: 0.77222
Orthogonal Random Features. Логистическая регрессия при new_dim > n_features. Доля верных ответов на тестовой выборке: 0.85490

CPU times: user 57.8 s, sys: 27.9 s, total: 1min 25s
Wall time: 2min 51s


In [420]:
#!c1.8
%%time
rffp_kernel = RFFPipeline(n_features=300, new_dim=500)
rffp_kernel.fit(x_train, y_train)
preds = rffp_kernel.predict(x_test)
print('Random Fourier Features. Логистическая регрессия при new_dim > n_features. Доля верных ответов на тестовой выборке: {:.5f}\n'.format(accuracy_score(preds, y_test)))

PCA применён
Медиана посчитана: 0.80286
Random Fourier Features. Логистическая регрессия при new_dim > n_features. Доля верных ответов на тестовой выборке: 0.84330

CPU times: user 59.7 s, sys: 26.2 s, total: 1min 25s
Wall time: 2min 56s


In [448]:
#!c1.8
%%time
rffp_kernel = ORFPipeline(n_features=500, new_dim=300)
rffp_kernel.fit(x_train, y_train)
preds = rffp_kernel.predict(x_test)
print('Orthogonal Random Features. Логистическая регрессия при new_dim < n_features. Доля верных ответов на тестовой выборке: {:.5f}\n'.format(accuracy_score(preds, y_test)))

PCA применён
Orthogonal Random Features. Логистическая регрессия при new_dim < n_features. Доля верных ответов на тестовой выборке: 0.86640

CPU times: user 59.4 s, sys: 22.9 s, total: 1min 22s
Wall time: 5min 17s


In [449]:
#!c1.8
%%time
rffp_kernel = RFFPipeline(n_features=500, new_dim=300)
rffp_kernel.fit(x_train, y_train)
preds = rffp_kernel.predict(x_test)
print('Random Fourier Features. Логистическая регрессия при new_dim < n_features. Доля верных ответов на тестовой выборке: {:.5f}\n'.format(accuracy_score(preds, y_test)))

PCA применён
Медиана посчитана: 0.78244
Random Fourier Features. Логистическая регрессия при new_dim < n_features. Доля верных ответов на тестовой выборке: 0.85970

CPU times: user 41.6 s, sys: 21.3 s, total: 1min 2s
Wall time: 4min 17s


### Сравнение качества Orthogonal Random Features и Random Fourier Features на линейном SVM

In [450]:
#!c1.8
%%time
linear_kernel = ORFPipeline(n_features=300, new_dim=500, classifier='svm')
linear_kernel.fit(x_train, y_train)
preds = linear_kernel.predict(x_test)
print('Orthogonal Random Features. Линейный SVM при new_dim > n_features. Доля верных ответов на тестовой выборке: {:.5f}\n'.format(accuracy_score(preds, y_test)))

PCA применён
Медиана посчитана: 0.79854
Orthogonal Random Features. Линейный SVM при new_dim > n_features. Доля верных ответов на тестовой выборке: 0.85270

CPU times: user 6min 58s, sys: 27.1 s, total: 7min 25s
Wall time: 6min 31s


In [451]:
#!c1.8
%%time
linear_kernel = RFFPipeline(n_features=300, new_dim=500, classifier='svm')
linear_kernel.fit(x_train, y_train)
preds = linear_kernel.predict(x_test)
print('Random Fourier Features. Линейный SVM при new_dim > n_features. Доля верных ответов на тестовой выборке: {:.5f}\n'.format(accuracy_score(preds, y_test)))

PCA применён
Медиана посчитана: 0.79058
Random Fourier Features. Линейный SVM при new_dim > n_features. Доля верных ответов на тестовой выборке: 0.85220

CPU times: user 8min 42s, sys: 25.6 s, total: 9min 7s
Wall time: 8min 15s


In [452]:
#!c1.8
%%time
linear_kernel = ORFPipeline(n_features=500, new_dim=300, classifier='svm')
linear_kernel.fit(x_train, y_train)
preds = linear_kernel.predict(x_test)
print('Orthogonal Random Features. Линейный SVM при new_dim < n_features. Доля верных ответов на тестовой выборке: {:.5f}\n'.format(accuracy_score(preds, y_test)))

PCA применён
Orthogonal Random Features. Логистическая регрессия при new_dim < n_features. Доля верных ответов на тестовой выборке: 0.86630

CPU times: user 13min 9s, sys: 20.9 s, total: 13min 30s
Wall time: 12min 51s


In [453]:
#!c1.8
%%time
linear_kernel = RFFPipeline(n_features=500, new_dim=300, classifier='svm')
linear_kernel.fit(x_train, y_train)
preds = linear_kernel.predict(x_test)
print('Random Fourier Features. Линейный SVM при new_dim < n_features. Доля верных ответов на тестовой выборке: {:.5f}\n'.format(accuracy_score(preds, y_test)))

PCA применён
Медиана посчитана: 0.75571
Random Fourier Features. Линейный SVM при new_dim < n_features. Доля верных ответов на тестовой выборке: 0.86330

CPU times: user 11min 48s, sys: 20.2 s, total: 12min 9s
Wall time: 11min 32s


**Вывод:** на обоих моделях: и линейном SVM и логистической регрессии при разных соотношениях new_dim <> n_features, как и ожидалось по результатам реализации алгоритма из статьи, Orthogonal Random Features показал лучшее качество, чем Random Fourier Features за счёт отсутствия у признаков высокой скоррелированности.